# AirBNB tracker

In [1]:
import numpy as np
import datetime
import pandas as pd
import pyperclip
import pickle
import re

In [53]:
tous_les_logements = pickle.load(open('tous_les_logements.p', 'rb'))
print(tous_les_logements.shape)
tous_les_logements.tail(2)

(127, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
149,2023-12-20,Appartement ⋅ Cachan\nAppartement lumineux\nAr...,None,None,Appartement lumineux,750,NaN,NaN,None,None,False,False
150,2023-12-20,Appartement ⋅ Bagneux\nBagneux YourHostHelper\...,None,None,Bagneux YourHostHelper,586,NaN,NaN,None,None,False,False


In [123]:
class Logement:
    __slots__ = [
        'date_enregistrement',
        'texte',
        'type',
        'ville',
        'description',
        'prix',
        'note',
        'nb_avis',
        'nb_lits',
        'nb_chambres',
        'est_professionnel',
        'est_nouveau'
    ]
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, est_nouveau: bool,
                 prix: int, note: float, nb_avis: int,
                 texte: str):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_professionnel = est_professionnel
        self.note = note
        self.nb_avis = nb_avis

        self.date_enregistrement = datetime.date.today()
        self.texte = texte
    
    @classmethod
    def from_text(cls, texte):
        texte = texte.strip()
        attrs = set(cls.__slots__) - {'texte', 'date_enregistrement'}
        kwargs = dict(texte=texte)
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[0]
        #return texte.split('\n')[0].split(pat)[0]
    
    @staticmethod
    def _get_ville(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[2]
        #return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str) -> tuple:
        nb_lits, nb_chambres = None, None
        pat = r'(?P<nb_lits>\d) lits?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_lits = int(rgx.group()[0])
        pat = r'(?P<nb_chambres>\d) chambres?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_chambres = int(rgx.group()[0])
        return nb_lits, nb_chambres
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    @staticmethod
    def _get_note(texte: str):
        return Logement._get_note_et_nb_avis(texte)[0]

    @staticmethod
    def _get_nb_avis(texte: str):
        return Logement._get_note_et_nb_avis(texte)[1]

    @staticmethod
    def _get_note_et_nb_avis(texte: str):
        note, nb_avis = None, None
        pat = r'(?P<note>\d\,\d\d?) \((?P<nb_avis>\d+)\)'
        if rgx := re.search(pat, texte):
            note = float(rgx.groupdict()['note'].replace(',', '.'))
            nb_avis = int(rgx.groupdict()['nb_avis'])
        return note, nb_avis
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep


def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    # stop = texte.find('1\n2\n3\n4')
    stop = re.search(r'[\d…]\n[\d…]\n[\d…]', texte)
    if stop:
        return texte[:stop.start()].strip()
    stop = re.search(r'Carte Google \d+ séjours affichés.', texte)
    return texte[:stop.start()].strip()

In [147]:
page = pyperclip.paste()

page = tronque_à_gauche(page)
page = tronque_à_droite(page)

In [148]:
pat = '(Appartement|Chambre|Hébergement|Tiny house|Appartement en résidence|Villa|Maison de ville) [⋅·] '
splits = [
    match.start()
    for match in re.finditer(pat, page)
]
splits += [len(page)]

logements = []
for texte in [page[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [149]:
dico = [{attr: getattr(logement, attr) for attr in Logement.__slots__} for logement in logements]
page_df = pd.DataFrame(dico, index=range(len(dico)))

In [150]:
page_df

,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
0,2023-12-25,Hébergement ⋅ Cachan\nMaison entière à 2km de ...,Hébergement,Cachan,Maison entière à 2km de Paris pour les JO 2024,705,NaN,NaN,None,None,False,False
1,2023-12-25,Appartement ⋅ Arcueil\nHome sweet home in Pari...,Appartement,Arcueil,Home sweet home in Paris,579,4.93,71.0,None,None,True,False
2,2023-12-25,Hébergement ⋅ Arcueil\nMaison cosy et jardin P...,Hébergement,Arcueil,Maison cosy et jardin Paris sud,708,4.90,21.0,None,None,False,False
3,2023-12-25,Hébergement ⋅ Arcueil\nLa Maison Ensoleillée -...,Hébergement,Arcueil,La Maison Ensoleillée - 2 Chambres - RER B & M7,1355,NaN,NaN,None,None,False,False
4,2023-12-25,Appartement ⋅ Arcueil\nAppartement duplex - Ar...,Appartement,Arcueil,Appartement duplex - Arcueil,565,4.87,30.0,None,None,False,False
5,2023-12-25,Appartement ⋅ Arcueil\nAppartement Cocon avec ...,Appartement,Arcueil,Appartement Cocon avec parking privé près de P...,690,NaN,NaN,None,None,False,False
6,2023-12-25,Appartement ⋅ Arcueil\nLa campagne à PARIS\nAr...,Appartement,Arcueil,La campagne à PARIS,584,4.69,26.0,None,None,False,False
7,2023-12-25,Appartement ⋅ Cachan\nL'appartement de Cachan\...,Appartement,Cachan,L'appartement de Cachan,339,NaN,NaN,None,None,False,False
8,2023-12-25,Appartement ⋅ Arcueil\nLe 11-26 Yourhosthelper...,Appartement,Arcueil,Le 11-26 Yourhosthelper,548,4.81,26.0,None,None,False,False
9,2023-12-25,Hébergement ⋅ Arcueil\nMaison d'Anne-Laure et ...,Hébergement,Arcueil,Maison d'Anne-Laure et Eric près de Paris,603,NaN,NaN,None,None,False,True


In [151]:
assert page_df.shape[0] == 18
assert page_df.drop(['nb_lits', 'nb_chambres'], axis=1).notnull().sum().min() > 0

In [152]:
# On ajoute à tous les logements la page en cours de scrapping : 
tous_les_logements = pd.concat([tous_les_logements, page_df], axis=0, ignore_index=True)
print(tous_les_logements.shape)
tous_les_logements.tail()

(307, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
302,2023-12-25,Hébergement ⋅ Arcueil\nMaison élégante près de...,Hébergement,Arcueil,Maison élégante près de Paris,447,NaN,NaN,None,None,False,False
303,2023-12-25,Appartement ⋅ Arcueil\nBeau T3 à Arcueil à 3 m...,Appartement,Arcueil,Beau T3 à Arcueil à 3 min RER B « Laplace »,573,4.83,54.0,None,None,False,False
304,2023-12-25,Chambre · Arcueil\nSéjournez chez Virginie Et ...,Chambre,Arcueil,Séjournez chez Virginie Et François,864,4.93,30.0,None,None,False,False
305,2023-12-25,Appartement ⋅ Gentilly\nAppartement familial\n...,Appartement,Gentilly,Appartement familial,550,NaN,NaN,None,None,False,False
306,2023-12-25,Maison de ville ⋅ Arcueil\nArcueil House (mais...,Maison de ville,Arcueil,Arcueil House (maison de ville),611,4.58,19.0,None,None,True,False


Quand on a fini d'itérer sur les pages, on dump :

In [153]:
pickle.dump(tous_les_logements.drop_duplicates().fillna(np.nan), open('tous_les_logements.p', 'wb'))